In [108]:
import os
import requests
from datetime import datetime


class Tools:
    def __init__(self):
        self.citation = True
        # pass

    def metadata_agent(self, message: str, __user__: dict = {}) -> str:
        """
        Send a message to the chat agent and concatenate the response messages into a single string.
        :param message: The message to send to the chat agent.
        :param session_id: The session ID for the chat.
        :return: A concatenated string of messages from the chat history.
        """

        def chat_with_agent(message: str, session_id: str = "test_session"):
            """Send a chat message to the agent endpoint"""

            url = "http://host.docker.internal:1234/api/v1/chat"

            payload = {
                "message": message,
                "session_id": session_id,
                "metadata": {"source": "test", "user_id": "test_user"},
            }

            headers = {"Content-Type": "application/json"}

            try:
                response = requests.post(url, json=payload, headers=headers)
                response.raise_for_status()
                response_data = response.json()

                print("\nResponse structure:")
                print(f"Keys in response: {response_data.keys()}")
                print(
                    f"Chat history length: {len(response_data.get('chat_history', []))}"
                )
                print(f"Final response: {response_data['response']['content']}")

                # Print chat history
                print("\nChat History:")
                for message in response_data.get("chat_history", []):
                    print(f"{message['name']}: {message['content']}")

                return response_data

            except requests.exceptions.RequestException as e:
                print(f"Error calling chat endpoint: {e}")
                if hasattr(e.response, "text"):
                    print(f"Error response: {e.response.text}")
                return None

        try:

            print(__user__)
            result = ""

            if "name" in __user__:
                result += f"User: {__user__['name']}"
            if "id" in __user__:
                result += f" (ID: {__user__['id']})"
            if "email" in __user__:
                result += f" (Email: {__user__['email']})"

            if result == "":
                result = "user_unknown"

            session_id = "test_session"

            response_data = chat_with_agent(message, session_id)
            if response_data and "chat_history" in response_data:
                print(response_data)
                concatenated = []
                for chat_message in response_data["chat_history"]:
                    role = chat_message.get("name", "Unknown Role")
                    content = chat_message.get("content", "")
                    concatenated.append(f"``````**{role}**``````: {content.replace('TERMINATE', '')}")
                print(concatenated)
                return "\n\n".join(concatenated)
            print("No chat history available.")
            return "No chat history available."
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return f"An error occurred: {str(e)}"
